In [1]:
from simtk import unit
from simtk import openmm

import numpy as np
from sys import stdout
from openmmtools import integrators
import random
import matplotlib.pyplot as plt

Defines system for use in BG and in MD simulation, can create from scratch as commented out

In [2]:
# pdb = app.PDBFile('ala2_fromURL.pdb')
# topology = pdb.getTopology()
# positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

# ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
# system = ff.createSystem(
#     topology=topology, 
#     removeCMMotion=True,
#     nonbondedMethod=app.NoCutoff,
#     constraints=app.HBonds, 
#     rigidWater=True
#     )

with open('proline/cis_pro_xmlsystem.txt') as f:
    xml = f.read()
system = openmm.XmlSerializer.deserialize(xml)
#platform 2 = CUDA
platform = openmm.Platform.getPlatform(2)

temperature_bg = 300.0 * unit.kelvin
collision_rate_bg = 1.0 / unit.picosecond
timestep_bg = 4.0 * unit.femtosecond

In [3]:
#Setting up generator
import torch

device = "cuda:3" if torch.cuda.is_available() else "cpu"
dtype = torch.float32
# a context tensor to send data to the right device and dtype via '.to(ctx)'
ctx = torch.zeros([], device=device, dtype=dtype)

#need to load a dataset for dimensions of BG and to set up Mixed Coordinate Transform which requires data as an argument
import mdtraj
rough_dataset_trans = mdtraj.load('proline/Trajectories/trans_pro_300K_long.dcd', top='proline/trans_pro_reindexed.pdb')
rough_dataset_cis = mdtraj.load('proline/Trajectories/cis_pro_300K_long.dcd',top='proline/cis_pro.pdb')

rough_dataset = rough_dataset_cis.join(rough_dataset_trans)

dataset = rough_dataset.superpose(rough_dataset_trans[0])

coordinates = dataset.xyz
n_atoms = 26

import numpy as np
rigid_block = np.array([9,7,6])
z_matrix = np.array([
    [0,4,6,7],
    [1,0,4,6],
    [2,0,4,6],
    [3,0,4,6],
    [4,6,7,9],
    [5,4,6,7],
    [8,7,6,4],
    [10,9,7,6],
    [11,7,9,24],
    [12,11,7,6],
    [13,11,7,6],
    [14,17,6,4],
    [15,14,17,6],
    [16,14,17,6],
    [17,6,7,9],
    [18,17,6,7],
    [19,17,6,7],
    [20,24,9,7],
    [21,20,24,9],
    [22,20,24,9],
    [23,20,24,9],
    [24,9,7,6],
    [25,24,9,7]
])


def dimensions(dataset):
        return np.prod(dataset.xyz[0].shape)
dim = dimensions(dataset)

#system setup, probably need to write a function to do this
# from simtk import openmm
# with open('ala2_xml_system.txt') as f:
#     xml = f.read()
# system = openmm.XmlSerializer.deserialize(xml)
from bgflow.distribution.energy.openmm import OpenMMBridge, OpenMMEnergy

integrator = integrators.LangevinIntegrator(temperature=temperature_bg,collision_rate=collision_rate_bg,timestep=timestep_bg)
energy_bridge = OpenMMBridge(system, integrator, n_workers=1)
target_energy = OpenMMEnergy(int(dim), energy_bridge)

#setting up training_data argument for MixedCoordinateTransform - not sure how much effect this has
n_train = len(dataset)//2
n_test = len(dataset) - n_train
permutation = np.random.permutation(n_train)

all_data = coordinates.reshape(-1, dimensions(dataset))
np.random.shuffle(all_data)
training_data = torch.tensor(all_data[permutation]).to(ctx)
test_data = torch.tensor(all_data[permutation + n_train]).to(ctx)

import bgflow as bg

dim_cartesian = len(rigid_block) * 3 - 6
dim_bonds = len(z_matrix)
dim_angles = dim_bonds
dim_torsions = dim_bonds

#set up coordinate transform layer
coordinate_transform = bg.MixedCoordinateTransformation(
    data=training_data, 
    z_matrix=z_matrix,
    fixed_atoms=rigid_block,
    keepdims=dim_cartesian, 
    normalize_angles=True,
).to(ctx)

#setting up prior distribution
dim_ics = dim_bonds + dim_angles + dim_torsions + dim_cartesian
mean = torch.zeros(dim_ics).to(ctx) 
# passing the mean explicitly to create samples on the correct device
prior = bg.NormalDistribution(dim_ics, mean=mean)

split_into_ics_flow = bg.SplitFlow(dim_bonds, dim_angles, dim_torsions, dim_cartesian)

#defining RealNVP
class RealNVP(bg.SequentialFlow):
    
    def __init__(self, dim, hidden):
        self.dim = dim
        self.hidden = hidden
        super().__init__(self._create_layers())
    
    def _create_layers(self):
        dim_channel1 =  self.dim//2
        dim_channel2 = self.dim - dim_channel1
        split_into_2 = bg.SplitFlow(dim_channel1, dim_channel2)
        
        layers = [
            # -- split
            split_into_2,
            # --transform
            self._coupling_block(dim_channel1, dim_channel2),
            bg.SwapFlow(),
            self._coupling_block(dim_channel2, dim_channel1),
            # -- merge
            bg.InverseFlow(split_into_2)
        ]
        return layers
        
    def _dense_net(self, dim1, dim2):
        return bg.DenseNet(
            [dim1, *self.hidden, dim2],
            activation=torch.nn.ReLU()
        )
    
    def _coupling_block(self, dim1, dim2):
        return bg.CouplingFlow(bg.AffineTransformer(
            shift_transformation=self._dense_net(dim1, dim2),
            scale_transformation=self._dense_net(dim1, dim2)
        ))

#setting up normalising flow composed of RealNVP followed by coordinate transform
n_realnvp_blocks = 5
layers = []

for i in range(n_realnvp_blocks):
    layers.append(RealNVP(dim_ics, hidden=[128, 128, 128]))
layers.append(split_into_ics_flow)
layers.append(bg.InverseFlow(coordinate_transform))

flow = bg.SequentialFlow(layers).to(ctx)

#loading trained model into empty
flow.load_state_dict(torch.load('proline/models/modeltrans_start_long_2.pt'))

#setting up generator
generator = bg.BoltzmannGenerator(
    flow=flow,
    prior=prior,
    target=target_energy)

In [4]:
#generator_new = bg.BoltzmannGenerator()

In [4]:
def getbg_positions(n_atoms):    
    bg_positions_tensor, dlogp_tensor = generator.sample(1,with_dlogp=True)
    bg_positions = bg_positions_tensor.cpu().detach().numpy().reshape(n_atoms,3)
    dlogp = dlogp_tensor.cpu().detach().numpy()
    #return bg_positions, np.exp(np.abs(dlogp))
    #return bg_positions, dlogp
    return bg_positions, np.abs(dlogp)
#print(bg_positions, dlogp)


In [17]:
print(getbg_positions(26))

(array([[ 4.7326293 , -0.23395705,  1.0906297 ],
       [ 4.7532744 , -0.19043311,  0.79869324],
       [ 4.87409   , -0.32564175,  1.3451197 ],
       [ 4.711592  , -0.01288781,  1.1687711 ],
       [ 4.6382403 , -0.3515219 ,  1.0805846 ],
       [ 4.662307  , -0.42342055,  0.98012316],
       [ 4.510528  , -0.3227    ,  1.1112814 ],
       [ 4.389726  , -0.37576294,  1.044713  ],
       [ 3.7838209 , -4.0896225 ,  2.4282722 ],
       [ 4.372186  , -0.36138007,  0.8897226 ],
       [ 4.389687  , -0.24792749,  0.8381607 ],
       [ 4.269942  , -0.3239477 ,  1.1293424 ],
       [ 2.444066  ,  4.035301  , -1.2536542 ],
       [ 1.5312198 , -3.3978136 ,  1.079711  ],
       [ 4.332883  , -0.2671749 ,  1.2536927 ],
       [ 0.5196426 ,  5.799145  ,  4.7965636 ],
       [ 4.3781714 , -2.9126058 ,  3.4488285 ],
       [ 4.4721227 , -0.23660576,  1.2163243 ],
       [ 4.565071  , -0.22968958,  1.3442065 ],
       [ 6.177704  ,  7.7791543 , -1.6977472 ],
       [ 4.1858234 , -0.5185547 ,  0.80

In [5]:
def getbias(positions,n_atoms):
    torch_positions = torch.tensor(positions.reshape(-1,n_atoms*3)).to(ctx)
    z, dlogp_inverse_tensor = flow.forward(torch_positions,inverse=True)
    dlogp_inverse = dlogp_inverse_tensor.cpu().detach().numpy()
    #return np.exp(np.abs(dlogp_inverse))
    return np.abs(dlogp_inverse)
    #return -dlogp_inverse

In [7]:
positions = generator.sample(1).cpu().detach().numpy().reshape(22,3)

print(getbias(positions,22))

[[334.7126]]


In [6]:
def getthermalenergy(temperature):
    #unit.BOLTZMANN_CONSTANT_kB is in units of J/K
    kb = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
    kt = kb * temperature
    kt = kt.value_in_unit(unit.kilojoule_per_mole)
    beta = 1/kt
    return beta

In [9]:
print(getthermalenergy(300*unit.kelvin))

0.40090785014242025


In [40]:
pdb = openmm.app.PDBFile('proline/cis_pro.pdb')
topology = pdb.getTopology()

positions = getbg_positions(26)[0]

md_temperature = 1000 * unit.kelvin
md_collision_rate = 10 / unit.picosecond
md_timestep = 1.0 * unit.femtosecond

integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "3"
simulation = openmm.app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
simulation.context.setPositions(positions)
first_state = simulation.context.getState(getPositions=True)
print(first_state.getPositions())
simulation.minimizeEnergy() 
second_state = simulation.context.getState(getPositions=True)
print(second_state.getPositions())
simulation.context.setVelocitiesToTemperature(md_temperature)
simulation.reporters.append(openmm.app.StateDataReporter(stdout, 
    reportInterval=100, 
    step=True, 
    totalEnergy = True,
    temperature=True,
))
#simulation.step(1000)

[Vec3(x=4.734786510467529, y=-0.36523401737213135, z=1.0350733995437622), Vec3(x=4.780878067016602, y=-0.4647982716560364, z=1.1607943773269653), Vec3(x=4.922467231750488, y=-0.2505784034729004, z=1.0247663259506226), Vec3(x=4.768473148345947, y=-0.3676775097846985, z=0.9165260791778564), Vec3(x=4.660071849822998, y=-0.26316219568252563, z=1.1150085926055908), Vec3(x=4.7333502769470215, y=-0.1796611249446869, z=1.170411229133606), Vec3(x=4.525393962860107, y=-0.2690741717815399, z=1.1238317489624023), Vec3(x=4.432069301605225, y=-0.35663050413131714, z=1.0531578063964844), Vec3(x=5.050313472747803, y=-4.9768147468566895, z=3.156266212463379), Vec3(x=4.431370258331299, y=-0.3478463888168335, z=0.8973779082298279), Vec3(x=4.467743396759033, y=-0.24396423995494843, z=0.8318742513656616), Vec3(x=4.290926933288574, y=-0.32159534096717834, z=1.1068693399429321), Vec3(x=1.4426606893539429, y=3.7879440784454346, z=-3.311846971511841), Vec3(x=1.7257803678512573, y=-4.783648490905762, z=1.420975

In [7]:
cycles = 1000
MDsteps = 1000
BGmoves = 1000

fname = 'coupled_pro_3000K_bias_long'

In [8]:
##Setting up MD and initialising

pdb = openmm.app.PDBFile('proline/cis_pro.pdb')
topology = pdb.getTopology()
positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)
n_atoms = positions.shape[0]

#reference frame for aligning all the MD conformations before pushing back through the flow to find the bias. Should come from BG training dataset - usually the first frame
reference_frame = mdtraj.load('proline/Trajectories/trans_pro_300K_long.dcd', top='proline/trans_pro_reindexed.pdb')[0]

md_temperature = 3000 * unit.kelvin
md_collision_rate = 10 / unit.picosecond
md_timestep = 1.0 * unit.femtosecond

parametersdict = {'Collision rate':md_collision_rate,'Temperature':md_temperature,'Timestep':md_timestep}
import pickle
with open(f'proline/parameters/parameters{fname}.pkl','wb') as f_p:
    pickle.dump(parametersdict,f_p)
f_p.close

integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
simulation = openmm.app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(md_temperature)
#simulation.reporters.append(openmm.app.DCDReporter(f'{fname}.dcd',100))
simulation.step(1000)
#simulation.reporters.pop(-1)

In [9]:
##WITH BIAS

#reporter for details about states - kinetic energy etc.
textfile_reporter = openmm.app.StateDataReporter(
    f'proline/Trajectories/{fname}.txt', 
    reportInterval=100, 
    step=True, 
    potentialEnergy=True,
    kineticEnergy = True,
    totalEnergy = True,
    temperature=True,
)

simulation.reporters.append(textfile_reporter)

#reporter to watch progress of simulation - to stdout
simulation.reporters.append(openmm.app.StateDataReporter(
    stdout, 
    reportInterval=100, 
    step=True, 
    totalEnergy = True,
    temperature=True,
))

#reporter for trajectory - positions
trajectory_reporter = openmm.app.DCDReporter(f'proline/Trajectories/{fname}.dcd',reportInterval=100)
simulation.reporters.append(trajectory_reporter)

accept_counter = 0
acceptance_probs = []

for x in range(cycles):
    print('cycle',x)
    simulation.step(MDsteps)
    current_state = simulation.context.getState(getEnergy=True,getPositions=True)
    current_positions = current_state.getPositions(asNumpy=True)
    current_positions_astraj = mdtraj.Trajectory(xyz=current_positions, topology=mdtraj.load('proline/cis_pro.pdb').topology)
    current_positions_aligned = current_positions_astraj.superpose(reference_frame).xyz
    bias_current = getbias(current_positions_aligned,n_atoms=n_atoms)
    #MDkinetic_energies.append(current_state.getKineticEnergy())
    #MDpotential_energies.append(current_state.getPotentialEnergy())
    current_total_energy = current_state.getKineticEnergy() + current_state.getPotentialEnergy()
    print('MD_end_energy',current_total_energy)
    for y in range(BGmoves):  
        integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
        bgsimulation = openmm.app.Simulation(topology,system,integrator,platform,platformProperties=properties_dict)
        bg_positions, bias_new = getbg_positions(n_atoms=n_atoms)
        bgsimulation.context.setPositions(bg_positions)
        bgsimulation.context.setVelocitiesToTemperature(md_temperature)
        new_state = bgsimulation.context.getState(getEnergy=True,getPositions=True)
        # only use this reporter if not breaking loop when new BG conformation is accepted
        # openmm.app.DCDReporter('coupled_BGonly.dcd',100, append=True).report(bgsimulation,new_state)
        new_total_energy = new_state.getKineticEnergy() + new_state.getPotentialEnergy()
        #print('new_tot_energy',new_total_energy)
        energy_change = (new_total_energy - current_total_energy).value_in_unit(unit.kilojoule_per_mole)
        bias_change = bias_current-bias_new
        acceptance_prob = min(1,(np.exp(bias_change-getthermalenergy(md_temperature)*energy_change)))
        #acceptance_prob = min(1,(np.exp(-getthermalenergy(md_temperature)*energy_change)))#*bias_current/bias_new))
        acceptance_probs.append(acceptance_prob)
        random_val = random.random()
        if random_val < acceptance_prob:
            print('accept new conformation')
            print('accepted BG energy',new_total_energy)
            #openmm.app.StateDataReporter(stdout,reportInterval=100, step=True, potentialEnergy=True,temperature=True).report(bgsimulation,new_state)
            #new_checkpoint = bgsimulation.context.createCheckpoint()
            #simulation.context.loadCheckpoint(new_checkpoint)
            BG_state = bgsimulation.context.getState(getEnergy=True,getPositions=True)
            trajectory_reporter.report(simulation,BG_state)
            textfile_reporter.report(simulation,BG_state)
            simulation.context.setState(BG_state)
            accept_counter += 1
            break
        else:
            print('rejected BG energy',y,new_total_energy)
        #simulation.context.setVelocitiesToTemperature(md_temperature)


cycle 0
#"Step","Total Energy (kJ/mole)","Temperature (K)"
1100,1670.5230102539062,3313.5412649854006
1200,1507.1053466796875,3590.049120968568
1300,1528.3641357421875,2657.5744561717797
1400,1405.7903442382812,2735.2152925964756
1500,1394.77099609375,3205.669637859687
1600,1227.0007934570312,2837.1521498334455
1700,1117.6871337890625,1561.9203352857082
1800,1389.9879150390625,2361.194339233386
1900,1201.4115600585938,2422.004670800429
2000,1649.330322265625,4120.836282044938
MD_end_energy 1649.330322265625 kJ/mol
rejected BG energy 0 1763.2947998046875 kJ/mol
rejected BG energy 1 1567.702880859375 kJ/mol
rejected BG energy 2 1565.4874267578125 kJ/mol
rejected BG energy 3 2308.1712036132812 kJ/mol
rejected BG energy 4 2050.3911743164062 kJ/mol
rejected BG energy 5 2509.053955078125 kJ/mol
rejected BG energy 6 1826.5870971679688 kJ/mol
rejected BG energy 7 1758.999755859375 kJ/mol
rejected BG energy 8 1872.7493896484375 kJ/mol
rejected BG energy 9 1201.1618041992188 kJ/mol
rejected BG e

In [ ]:
##ALTERED VERSION WITH POT. ENERGY COMPARISON
simulation.reporters.append(openmm.app.StateDataReporter(
    f'{fname}.txt', 
    reportInterval=100, 
    step=True, 
    potentialEnergy=True,
    kineticEnergy = True,
    totalEnergy = True,
    temperature=True,
))
simulation.reporters.append(openmm.app.StateDataReporter(
    stdout, 
    reportInterval=100, 
    step=True, 
    totalEnergy = True,
    temperature=True,
))
simulation.reporters.append(openmm.app.DCDReporter(f'{fname}.dcd',reportInterval=100))#, #append=True))

for x in range(cycles):
    print('cycle',x)
    simulation.step(MDsteps)
    current_state = simulation.context.getState(getEnergy=True,getPositions=True)
    current_positions = current_state.getPositions(asNumpy=True)
    current_positions_astraj = mdtraj.Trajectory(xyz=current_positions, topology=mdtraj.load('ala2_fromURL.pdb').topology)
    current_positions_aligned = current_positions_astraj.superpose(reference_frame).xyz
    bias_current = getbias(current_positions_aligned,n_atoms=n_atoms)
    #MDkinetic_energies.append(current_state.getKineticEnergy())
    #MDpotential_energies.append(current_state.getPotentialEnergy())
    current_pot_energy = current_state.getPotentialEnergy()
    print('MD_end_energy',current_pot_energy)
    for y in range(BGmoves):  
        integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
        bgsimulation = openmm.app.Simulation(topology,system,integrator,platform,platformProperties=properties_dict)
        bg_positions, bias_new = getbg_positions(n_atoms=n_atoms)
        bgsimulation.context.setPositions(bg_positions)
        new_state = bgsimulation.context.getState(getEnergy=True,getPositions=True)
        new_pot_energy = new_state.getPotentialEnergy()
        #print('new_tot_energy',new_total_energy)
        energy_change = (new_pot_energy - current_pot_energy).value_in_unit(unit.kilojoule_per_mole)
        acceptance_prob = min(1,(np.exp(-getthermalenergy(md_temperature)*energy_change)))#*bias_current/bias_new))
        random_val = random.random()
        if random_val < acceptance_prob:
            print('accept new conformation')
            print('accepted BG energy',new_pot_energy)
            #openmm.app.StateDataReporter(stdout,reportInterval=100, step=True, potentialEnergy=True,temperature=True).report(bgsimulation,new_state)
            new_checkpoint = bgsimulation.context.createCheckpoint()
            simulation.context.loadCheckpoint(new_checkpoint)
            #simulation.context.getState(getPositions=True)
            #openmm.app.DCDReporter('coupled.dcd',100, append=True).report(simulation,)
            break
        else:
            print('rejected BG energy',y,new_pot_energy)


In [ ]:
integrator = integrators.LangevinIntegrator(temperature=md_temperature,collision_rate=md_collision_rate,timestep=md_timestep)
bgsimulation = openmm.app.Simulation(topology,system,integrator,platform,platformProperties=properties_dict)
bg_positions, bias_new = getbg_positions(n_atoms=n_atoms)
bgsimulation.context.setPositions(bg_positions)
new_state = bgsimulation.context.getState(getEnergy=True,getPositions=True)
new_pot_energy = new_state.getPotentialEnergy()
#print('new_tot_energy',new_total_energy)
energy_change = (new_pot_energy - current_pot_energy).value_in_unit(unit.kilojoule_per_mole)
acceptance_prob = min(1,(np.exp(-getthermalenergy(md_temperature)*energy_change)))#*bias_current/bias_new))
random_val = random.random()
if random_val < acceptance_prob:
    print('accept new conformation')
    print('accepted BG energy',new_pot_energy)
    #openmm.app.StateDataReporter(stdout,reportInterval=100, step=True, potentialEnergy=True,temperature=True).report(bgsimulation,new_state)
    new_checkpoint = bgsimulation.context.createCheckpoint()
    simulation.context.loadCheckpoint(new_checkpoint)

In [10]:
#print(accept_counter /(BGmoves*cycles))
print(np.average(acceptance_probs))
#print(np.average(BGkinetic_energies))
#print(np.average(BGpotential_energies))

0.13593952522081684


In [12]:
import mdtraj as md

#traj = md.load('pro_trajectory.dcd',top=md.Topology.from_openmm(topology))
traj = md.load('coupled.dcd',top='ala2_fromURL.pdb')
print
#print(traj[24].xyz)
import nglview
view = nglview.show_mdtraj(traj)
view

In [ ]:
reference_frame = mdtraj.load('TSFtraj.dcd',top='ala2_fromURL.pdb')[0]

In [ ]:
simulation.step(MDsteps)
current_state = simulation.context.getState(getEnergy=True,getPositions=True)
current_positions = current_state.getPositions(asNumpy=True)
current_positions_astraj = mdtraj.Trajectory(xyz=current_positions, topology=mdtraj.load('ala2_fromURL.pdb').topology)
current_positions_aligned = current_positions_astraj.superpose(reference_frame).xyz
bias_current = getbias(current_positions_aligned)
print(bias_current)
current_total_energy = current_state.getKineticEnergy() + current_state.getPotentialEnergy()
print('MD_end_energy',current_total_energy)

In [ ]:
#current_positions = current_state.getPositions(asNumpy=True)
torch_positions = torch.tensor(current_positions_aligned.reshape(-1,66)).to(ctx)

z = flow.forward(torch_positions,inverse=True)
x_out = flow.forward(z[0])
z_2 = flow.forward(x_out[0],inverse=True)
x_2 = flow.forward(z_2[0])
z_3 = flow.forward(x_out[0],inverse=True)
x_3 = flow.forward(z_3[0])
print('initial_x',torch_positions)
print('z 1',z)
print('x_out',x_out)
print('z 2',z_2)
print('x_2',x_2)
print('z 3',z_3)
print('x_3',x_3)

In [ ]:
bg_positions, z = generator.sample(1, with_latent=True)
z_from_inverse = flow.forward(bg_positions, inverse=True)

print(bg_positions)
print(z)
print(z_from_inverse)

x_new = flow.forward(z_from_inverse[0])
x_fromlatent = flow.forward(z)

print(x_new)
print(x_fromlatent)

In [ ]:
BG_cartesian = generator.sample(1000)

MDTSF_traj = mdtraj.load('TSFtraj.dcd',top='ala2_fromURL.pdb',stride=1000)
MDTSF_cartesian = torch.tensor(MDTSF_traj.xyz.reshape(-1,66)).to(ctx)

MD3000K_traj = mdtraj.load('3000K.dcd',top='ala2_fromURL.pdb',stride=100)
MD3000K_cartesian = torch.tensor(MD3000K_traj.xyz.reshape(-1,66)).to(ctx)

BG300K_traj = mdtraj.load('TSFtraj_xmlsystem_20000KLL_samplestraj.dcd',top='ala2_fromURL.pdb',stride=10)
BG300K_cartesian = torch.tensor(BG300K_traj.xyz.reshape(-1,66)).to(ctx)

MD300K_superposed_traj = MD300K_traj.superpose(MD300K_traj[0])
MD300K_superposed_cartesian = torch.tensor(MD300K_superposed_traj.xyz.reshape(-1,66)).to(ctx)

MD1000K_traj = mdtraj.load('1000K_2.dcd',top='ala2_fromURL.pdb',stride=100)
MD1000K_cartesian = torch.tensor(MD1000K_traj.xyz[0:1000].reshape(-1,66)).to(ctx)

MD1000K_superposed_traj = MD1000K_traj.superpose(MD1000K_traj[0])
MD1000K_superposed_cartesian = torch.tensor(MD300K_superposed_traj.xyz.reshape(-1,66)).to(ctx)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8,4))
means = {}
dist_from_means = {}
cartesian_sets = {'BG 20000KLL on 1000 K traj': BG_cartesian,
            'MD TSF': MDTSF_cartesian,
            #'MD1000K' : MD1000K_cartesian,
            #'MD300K superposed' : MD300K_superposed_cartesian,
            'MD1000K superposed' : MD1000K_superposed_cartesian,
            #'MD non superposed 3000 K' : MD3000K_cartesian,
            'BG 20000 KLL trained TSF trajectory' : BG300K_cartesian
            }

names_list = list(cartesian_sets)
for name, cartesian_data in cartesian_sets.items():
    means[name] = torch.mean(cartesian_data, dim = 0)
    dist_from_means[name] = torch.sub(cartesian_data, means[name])
    ax.hist(dist_from_means[name].flatten().cpu().detach().numpy(), bins = 40, label=f'{name}', alpha = (1-names_list.index(name)/5))

ax.legend(bbox_to_anchor = (1.04,1), loc='upper left')
ax.set_xlabel("Distance from mean, Cartesian space")
#ax.set_ylabel(f"Count   [#Samples / {len(cartesian_data[name].flatten())}]")

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
latents = {}
means = {}
dist_from_means = {}
# cartesian_sets = {'BG 2000KLL trained on TSF traj': BG_cartesian,
#             #'MD300K': MD300K_cartesian,
#             #'MD1000K' : MD1000K_cartesian,
#             #'MD300K superposed' : MD300K_superposed_cartesian,
#             'MD non superposed 3000 K' : MD3000K_cartesian,
#             'BG 2000 KLL trained on superposed 3000 K' : BG3000K_cartesian
#             }

names_list = list(cartesian_sets)
for name, cartesian_data in cartesian_sets.items():
    latents[cartesian_data] = flow.forward(cartesian_data,inverse=True)[0]
    means[cartesian_data] = torch.mean(latents[cartesian_data], dim = 0)
    dist_from_means[cartesian_data] = torch.sub(latents[cartesian_data], means[cartesian_data])
    ax.hist(dist_from_means[cartesian_data].flatten().cpu().detach().numpy(), bins = 100, label=f'{name}', alpha = (1-names_list.index(name)/5))

ax.legend(bbox_to_anchor = (1.04,1), loc='upper left')
ax.set_xlabel("Distance from mean, latent space")
#ax.set_ylabel(f"Count   [#Samples / {len(BG_cart_distfrommean.flatten())}]")

plt.show()